### The `__prepare__` Method

We know that when we create a class, the metaclass `__new__` method is invoked with an argument (`cls_dict`) for the class dictionary.

It is not in fact an empty dictionary at first:

In [7]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('MyMeta.__new__ called...')
        print('\tcls: ', mcls, type(mcls))
        print('\tname:', name, type(name))
        print('\tbases: ', bases, type(bases))
        print('\tcls_dict:', cls_dict, type(cls_dict))
        print('\tkwargs:', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [8]:
class MyClass(metaclass=MyMeta):
    pass

MyMeta.__new__ called...
	cls:  <class '__main__.MyMeta'> <class 'type'>
	name: MyClass <class 'str'>
	bases:  () <class 'tuple'>
	cls_dict: {'__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
	kwargs: {}


# So, as we see, `cls_dict` is a dictionary and it also contains some information already. It is obviously being created somewhere before being passed to the `__new__` method.

The class dictionary is actually created by calling the `__prepare__` method, which the `type` class implements.

When the class is created, Python calls `__prepare__` and uses the return value of that method as the initialized class dictionary.
Then right before calling `__new__` it adds a few items into that dictionary, and then calls the `__new__` method using that pre-created and initialized dictionary.

Since `__prepare__` is just a method in `type`, we can override it.

In [10]:
class MyMeta(type):
    @staticmethod
    def __prepare__(name, bases, **kwargs):
        print('MyMeta.__prepare__ called...')
        print('\tname:', name)
        print('\tkwargs:', kwargs)
        return {'a': 100, 'b': 200}
    
    @staticmethod
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('MyMeta.__new__ called...')
        print('\tcls: ', mcls, type(mcls))
        print('\tname:', name, type(name))
        print('\tbases: ', bases, type(bases))
        print('\tcls_dict:', cls_dict, type(cls_dict))
        print('\tkwargs:', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [12]:
class MyClass(metaclass=MyMeta, kw1=11, kw2=22):
    pass

MyMeta.__prepare__ called...
	name: MyClass
	kwargs: {'kw1': 11, 'kw2': 22}
MyMeta.__new__ called...
	cls:  <class '__main__.MyMeta'> <class 'type'>
	name: MyClass <class 'str'>
	bases:  () <class 'tuple'>
	cls_dict: {'a': 100, 'b': 200, '__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
	kwargs: {'kw1': 11, 'kw2': 22}


Notice how the `__prepare__` method was called **before** the `__new__` method was called.

Also notice how it contains the items `'a': 100` and `'b': 200` which we injected in the `__prepare__` method.

The `cls_dict` argument in `__new__` has a couple of extra items that it injects for us prior to calling the `__new__` method.

Of course, if we do not specify a `__prepare__` method in our metaclass, we inherit the one that is already defined in `type` - which returns an empty dictionary.

In [13]:
type.__prepare__()

{}

Here's an example where using this method can simplify things somewhat.

Recall the example where we passed named arguments to the metaclass in order to create some additional class attributes:

In [6]:
class MyMeta(type):
    def __new__(mcls, name, bases, class_dict, **kwargs):
        class_dict.update(kwargs)
        return super().__new__(mcls, name, bases, class_dict)
    
class MyClass(metaclass=MyMeta, arg1=100, arg2=200):
    pass        

In [7]:
vars(MyClass)

mappingproxy({'__module__': '__main__',
              'arg1': 100,
              'arg2': 200,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

We were able to override the `__new__` method and inject the additional arguments right into the class dictionary.

But we could just as easily inject those items in the class dictionary right in the `__prepare__` method.

In [19]:
class MyMeta(type):
    def __prepare__(name, bases, **kwargs):
        print(f'MyMeta.__prepare__ called... with {kwargs}')
        # we could create a new dictionary and insert everything we need from kwargs
        # or we could just use the kwargs dictionary directly
        kwargs['bonus_attr'] = 'Python rocks!'


        return kwargs
    
class MyClass(metaclass=MyMeta, kw1=1, kw2=2): # so we have t specify the new
    pass

MyMeta.__prepare__ called... with {'kw1': 1, 'kw2': 2}


TypeError: MyClass.__init_subclass__() takes no keyword arguments

What's important to understand is that whatever extra arguments we pass to the metaclass are also passed along to the `__prepare__` method, just like they are eventually passed to `__new__`.

In [14]:
class MyMeta(type):
    def __prepare__(name, bases, **kwargs):
        print(f'MyMeta.__prepare__ called... with {kwargs}')
        # we could create a new dictionary and insert everything we need from kwargs
        # or we could just use the kwargs dictionary directly
        kwargs['bonus_attr'] = 'Python rocks!'
        return kwargs
    
    def __new__(cls, name, bases, cls_dict, **kwargs):
        print('MyMeta.__new__ called...')
        print('\tcls: ', cls, type(cls))
        print('\tname:', name, type(name))
        print('\tbases: ', bases, type(bases))
        print('\tcls_dict:', cls_dict, type(cls_dict))
        print('\tkwargs:', kwargs)
        return super().__new__(cls, name, bases, cls_dict)

In [15]:
class MyClass(metaclass=MyMeta, kw1=1, kw2=2):
    pass

MyMeta.__prepare__ called... with {'kw1': 1, 'kw2': 2}
MyMeta.__new__ called...
	cls:  <class '__main__.MyMeta'> <class 'type'>
	name: MyClass <class 'str'>
	bases:  () <class 'tuple'>
	cls_dict: {'kw1': 1, 'kw2': 2, 'bonus_attr': 'Python rocks!', '__module__': '__main__', '__qualname__': 'MyClass'} <class 'dict'>
	kwargs: {'kw1': 1, 'kw2': 2}


In [16]:
vars(MyClass)

mappingproxy({'kw1': 1,
              'kw2': 2,
              'bonus_attr': 'Python rocks!',
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

And as you can see, we have our class attributes, and we did not have to use `__new__`. So often, `__prepare__` is a much simpler alternative to overriding `__new__`.

The return value of `__prepare__` must be a mapping type:

In [20]:
class MyMeta(type):
    def __prepare__(name, bases):
        return 'some string'

In [21]:
class MyClass(metaclass=MyMeta):
    pass

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mohammedhamdan/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/gk/qmm3q5ld1b74kcdmh5pnm1xw0000gn/T/ipykernel_91117/282976637.py", line 1, in <module>
    class MyClass(metaclass=MyMeta):
  File "/var/folders/gk/qmm3q5ld1b74kcdmh5pnm1xw0000gn/T/ipykernel_91117/282976637.py", line 1, in MyClass
    class MyClass(metaclass=MyMeta):
TypeError: string indices must be integers

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mohammedhamdan/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/mohammedhamdan/Library/Python/3.10/lib/python/site-packages/IPython/core/ultratb.py", line 1112, in structured_traceback
    return Forma

This exception is raised because Python is trying to use the class dictionary as a mapping type.

In [23]:
cls_dict = 'some string'
cls_dict['__module__']

TypeError: string indices must be integers

The return value must therefore be a mapping type, but it does not have to be a dict - it could be an OrderedDict for example, or even a custom dictionary.

In [24]:
from collections import OrderedDict

In [15]:
class MyMeta(type):
    def __prepare__(name, bases):
        d = OrderedDict()
        d['bonus'] = 'Python rocks!'
        return d

In [26]:
class MyClass(metaclass=MyMeta):
    pass

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/mohammedhamdan/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/gk/qmm3q5ld1b74kcdmh5pnm1xw0000gn/T/ipykernel_91117/282976637.py", line 1, in <module>
    class MyClass(metaclass=MyMeta):
  File "/var/folders/gk/qmm3q5ld1b74kcdmh5pnm1xw0000gn/T/ipykernel_91117/282976637.py", line 1, in MyClass
    class MyClass(metaclass=MyMeta):
TypeError: string indices must be integers

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mohammedhamdan/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/Users/mohammedhamdan/Library/Python/3.10/lib/python/site-packages/IPython/core/ultratb.py", line 1112, in structured_traceback
    return Forma

In [27]:
vars(MyClass)

mappingproxy({'bonus_attr': 'Python rocks!',
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

Or it could even be a custom dictionary type:

In [28]:
from collections import UserDict

In [30]:
class CustomDict(UserDict):
    def __setitem__(self, key, value):
        print(f'Setting {key} = {value} in custom dictionary')
        super().__setitem__(key, value)
        
    def __getitem__(self, key):
        print(f'Getting {key} from custom dictionary')
        return int(super().__getitem__(key))   

In [31]:
class MyMeta(type):
    def __prepare__(name, bases):
        return CustomDict()

In [33]:
class MyClass(metaclass=MyMeta):
    pass

Getting __name__ from custom dictionary
Setting __module__ = __main__ in custom dictionary
Setting __qualname__ = MyClass in custom dictionary


TypeError: type.__new__() argument 3 must be dict, not CustomDict

In [34]:
issubclass(UserDict, dict)

False

As you can see, we have a slight problem here. The `__new__` method actually expects a `dict`. Even though `CustomDict` essentially behaves like a dictionary, it is not in fact a subclass of `dict`:

In [22]:
issubclass(CustomDict, dict)

False

But as long as our custom dictionary inherits from `dict` we should be fine:

In [35]:
class CustomDict(dict):
    def __setitem__(self, key, value):
        print(f'Setting {key} = {value} in custom dictionary')
        super().__setitem__(key, value)
        
    def __getitem__(self, key):
        print(f'Getting {key} from custom dictionary')
        return int(super().__getitem__(key))   

In [36]:
class MyMeta(type):
    def __prepare__(name, bases):
        return CustomDict()
    
    def __new__(mcls, name, bases, cls_dict):
        print('metaclass __new__ called...')
        print(f'\ttype(cls_dict) = {type(cls_dict)}')
        print(f'\tcls_dict={cls_dict}')

In [37]:
class MyClass(metaclass=MyMeta):
    pass

Getting __name__ from custom dictionary
Setting __module__ = __main__ in custom dictionary
Setting __qualname__ = MyClass in custom dictionary
metaclass __new__ called...
	type(cls_dict) = <class '__main__.CustomDict'>
	cls_dict={'__module__': '__main__', '__qualname__': 'MyClass'}


As you can see, the dictionary we returned from `__prepare__` was a `CustomDict` instance that is eventually passed to `__new__` when it is called. 

And between `__prepare__` and `__new__`, Python accessed our dictionary to read/write a few items.